# Generate object(sign) and annotations on draw

In [93]:
%matplotlib inline

import os
from glob import glob
from matplotlib import pyplot
from PIL import Image
import numpy as np
import xml.etree.ElementTree as ET
import random
import cv2

In [87]:
draw_data = glob('./data/backGround/draw/*.png')
annotaions_data = glob('./data/backGround/annotations/*.xml')
sign_data = glob('./data/backGround/sign/*.png')

# Reshape Sign 
size = 15, 15

for s in sign_data:
    im = Image.open(s)
    im.thumbnail(size, Image.ANTIALIAS)
    im.save(s)

# Make Xml File

In [ ]:
def makeXml(x1,x2,y1,y2, dr_fileName):
    
    root = ET.Element("root")
    doc = ET.SubElement(root, "doc")

    ET.SubElement(doc, "field1", name="blah").text = "some value1"
    ET.SubElement(doc, "field2", name="asdfasd").text = "some vlaue2"

    tree = ET.ElementTree(root)
    tree.write("filename.xml")


In [ ]:
lower_blue = np.array([110,50,50])
upper_blue = np.array([130,255,255])

mask = cv2.inRange(hsv, lower_blue, upper_blue)
res = cv2.bitwise_and(15,15 , mask= mask)


# Change color of sign

In [77]:
input_image = cv2.imread('./data/backGround/sign/DT_131.png',-1)

bl_lower= (0, 0, 0) #BGR순서
bl_upper= (0, 0, 0)


mask = cv2.inRange(input_image, bl_lower, bl_upper)
input_image[mask>0]=(52,55,98)

cv2.imwrite('./data/backGround/exS.png', input_image) 

  

error: OpenCV(4.1.2) /io/opencv/modules/core/src/arithm.cpp:1742: error: (-209:Sizes of input arguments do not match) The lower boundary is neither an array of the same size and same type as src, nor a scalar in function 'inRange'


In [84]:
input_image = cv2.imread('./data/backGround/sign/DT_131.png',-1)
input_image=cv2.cvtColor(input_image,cv2.COLOR_BGR2HSV)
ret, mask = cv2.threshold(input_image, 0, 255,cv2.THRESH_BINARY_INV |cv2.THRESH_OTSU)

input_image.Set(mask,cv2.Scalar(0,0,255))


cv2.imwrite('./data/backGround/exS.png', input_image) 


True

# Make n-Sign on Draw with loaction(x,y) information 

In [89]:

def makeSign(clss, amount, dr_fileName):
    
    l_img =  cv2.imread(dr_fileName)
    s_img =  cv2.imread('./data/backGround/sign/'+clss+'.png',-1) 
    
    dr_fileName = dr_fileName.split('draw/')[1]
    dr_fileName = dr_fileName.split('.png')[0]    
    
    new_fileName = './data/draw/dataset/GN_'+dr_fileName+int(radom.random()*6)+'.png'   

    # amount 수만큼 기호 생성.                 
    for k in amount:    
        xmin = random.sample(range(100,width))
        ymin = random.sample(range(100,height))
                
        x_offset=y_offset=16
        
        x1, x2 = xmin, x_offset + s_img.shape[1]
        y1, y2 = ymin, y_offset + s_img.shape[0]

        alpha_s = s_img[:, :, 3] / 255.0
        alpha_l = 1.0 - alpha_s
 
    for c in range(0, 3):
        l_img[y1:y2, x1:x2, c] = (alpha_s * s_img[:, :, c] + alpha_l * l_img[y1:y2, x1:x2, c])
        
        
    
 
    cv2.imwrite(new_fileName, l_img)   

In [49]:
         


l_img = cv2.imread('./data/backGround/draw/10f.png')
s_img = cv2.imread('./data/backGround/sign/DT_131.png',-1)
                 
        
x_offset=y_offset=16
        
y1, y2 = y_offset, y_offset + s_img.shape[0]
x1, x2 = x_offset, x_offset + s_img.shape[1]

alpha_s = s_img[:, :, 3] / 255.0
alpha_l = 1.0 - alpha_s

for c in range(0, 3):
    l_img[y1:y2, x1:x2, c] = (alpha_s * s_img[:, :, c] +  alpha_l * l_img[y1:y2, x1:x2, c])

    
cv2.imwrite('./data/backGround/temp.png', l_img)      

True

In [ ]:
count = 0

for clss in root.iter('name'):
    count+=1
    if clss.text == ''
    print(clss.text)

print(count)